**ColorClustering**

Compress an image by:
1. Clustering RGB values of pixels (using K-Means)
2. Replace pixel with an average color of it's cluster

[Credit](https://www.youtube.com/watch?v=yR7k19YBqiw)

In [ ]:
import numpy as np
import pandas as pd

from PIL import Image
from sklearn.cluster import KMeans

In [ ]:
class MyImage:
    
    def __init__(self, path):
        
        # open image
        self.image = Image.open(path)
        self.rgb_pixels = np.array(self.image)
        self.original_shape = self.rgb_pixels.shape
        
        # get (red, green, blue) layers
        r, g, b = np.transpose(
            self.rgb_pixels, 
            axes=(2, 0, 1)
        )
        
        # flatten and convert to df (for clustering)
        self.df = pd.DataFrame(dict(
            r=r.flatten(),
            g=g.flatten(),
            b=b.flatten()
        ))
        
    
    def get_compressed_image(self, 
                             n_clusters=10, 
                             sample_size=10_000):
        
        clustering = KMeans(n_clusters=n_clusters)
        clustering.fit(self.df.sample(sample_size))  # fitting on subsample to speed up
        
        clusters = clustering.predict(self.df)
        clusters = pd.Series(clusters)  # to use .map() later
        
        group_means = self.df.groupby(clusters).mean()
        group_means = group_means.to_dict()
        
        df_new = pd.DataFrame()
        for c in self.df.columns:
            df_new[c] = clusters.map(group_means[c])
        df_new = df_new.astype(np.uint8)  # to convert back to image
        
        rgb_array = df_new.values.reshape(self.original_shape)
        return Image.fromarray(rgb_array)
    
    
    def save_compressed_images(self, 
                               k_colors, 
                               output_folder='output'):
        
        for k in k_colors:
            image = self.get_compressed_image(n_clusters=k)
            image.save(f'{output_folder}/k_{k:02}.jpg')
        
        self.image.save(f'{output_folder}/k_original.jpg')  # to compare with original image

In [ ]:
image = MyImage('test_image.jpg')

In [ ]:
image.save_compressed_images(k_colors=[2,3,4,5,6])